In [1]:
# Import libraries

import sys
import logging
import pandas as pd
import xarray as xr
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from library.assumptions import read_assumptions
from library.weather import load_weather
from library.network import build_network

logging.basicConfig(level=logging.INFO)

In [2]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    "EUR": 10.5080,
    "USD": 9.2521
}
base_year = 2024
discount_rate = 0.04
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3

## Parameters that will change frequently
target_year = 2030
use_offwind = True
use_h2 = True
h2_initial = 1000
biogas_limit = 0
load_target = 0.1 #TWh

In [3]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_path / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the normalized demand from csv file (see normalize_demand() in library.demand for details)
# And then calculate target_load using projection of energy need in target_year
normalized_demand = pd.read_csv(paths.input_path / 'demand/normalized-demand-2023-3h.csv', delimiter=',')
target_load = load_target * normalized_demand['value'].values.flatten() * 1_000_000

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
geo = '14' # All of VGR
section = None
cutout, selection, index = load_weather(geo, section, '2023-01', '2023-12')
geography = selection.total_bounds  

capacity_factor_solar = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-solar.nc").values.flatten()
capacity_factor_onwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-onwind.nc").values.flatten()
capacity_factor_offwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-offwind.nc").values.flatten()

In [4]:
# Build the network

network = build_network(index, resolution, geography, target_load, assumptions, capacity_factor_solar, capacity_factor_onwind, capacity_factor_offwind, use_offwind, use_h2, h2_initial, biogas_limit)

In [5]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

## Add offwind constraint
if False:
    generator_capacity = model.variables["Generator-p_nom"]
    offwind_percentage = 0.5

    offwind_constraint = (1 - offwind_percentage) / offwind_percentage * generator_capacity.loc['offwind'] - generator_capacity.loc['onwind']
    model.add_constraints(offwind_constraint == 0, name="Offwind_constraint")

## Add battery charge/discharge ratio constraint
link_capacity = model.variables["Link-p_nom"]
lhs = link_capacity.loc["battery-charge"] - network.links.at["battery-charge", "efficiency"] * link_capacity.loc["battery-discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing integer variables.: 100%|██████████| 2/2 [00:00<00:00, 848.88it/s]
INFO:linopy.io: Writing time: 0.47s
INFO:linopy.solvers:Log file at /tmp/highs.log


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 40897 primals, 90538 duals
Objective: 1.28e+08
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


Coefficient ranges:
  Matrix [4e-06, 1e+02]
  Cost   [6e+01, 2e+06]
  Bound  [0e+00, 0e+00]
  RHS    [7e+00, 2e+01]
Presolving model
45264 rows, 36512 cols, 118272 nonzeros  0s
37991 rows, 29239 cols, 111910 nonzeros  0s
37991 rows, 29239 cols, 111910 nonzeros  0s

Solving MIP model with:
   37991 rows
   29239 cols (0 binary, 5 integer, 0 implied int., 29234 continuous)
   111910 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.2s
 S       0       0         0   0.00%   -inf            132309809.718      Large        0      0      0         0     5.5s
 R       0       0         0   0.00%   128071320.778   128507247.8961     0.34%        0      0      0     40876     

('ok', 'optimal')

In [6]:
network.statistics()

Optimal Capacity  Installed Capacity        Supply  \
Generator backstop         16.835097                 0.0   3490.309903   
          onwind           47.057143                 0.0  47218.091993   
          solar            52.875060                 0.0  51294.749031   
Link      AC               33.540790                 0.0  97524.097821   
          li-ion           15.977255                 0.0  22210.573001   
Load      -                 0.000000                 0.0      0.000000   
Store     li-ion          171.068324                 0.0  21938.333701   

                      Withdrawal       Dispatch  Transmission  \
Generator backstop  0.000000e+00    3490.309903      0.000000   
          onwind    7.993606e-15   47218.091993      0.000000   
          solar     0.000000e+00   51294.749031      0.000000   
Link      AC        9.851284e+04    -988.743203  74299.117096   
          li-ion    2.315613e+04    -945.557430      0.000000   
Load      -         1.000000e+05 -100000.000000      0.000000   
Store     li-ion    2.200718e+04     -68.850294      0.000000   

                    Capacity Factor    Curtailment  Capital Expenditure  \
Generator backstop         0.023667       0.000000         0.000000e+00   
          onwind           0.114546  110555.572116         4.694861e+07   
          solar            0.110743    8436.699431         2.032800e+07   
Link      AC               0.335286       0.000000         2.420428e+06   
          li-ion           0.165447       0.000000         0.000000e+00   
Load      -                     NaN       0.000000         0.000000e+00   
Store     li-ion           0.574350       0.000000         2.438529e+07   

                    Operational Expenditure  Revenue  Market Value  
Generator backstop             3.300856e+07      0.0           NaN  
          onwind               1.005803e+06      0.0           NaN  
          solar                0.000000e+00      0.0           NaN  
Link      AC                   0.000000e+00      0.0           NaN  
          li-ion               0.000000e+00      0.0           NaN  
Load      -                    0.000000e+00      0.0           0.0  
Store     li-ion              -1.429594e+03      0.0           NaN